<a href="https://colab.research.google.com/github/you444Mo/Python-Projects/blob/main/GB656_Week_3_Assignment_(Baseball_GLM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Baseball Runs Predictive Model

**Baseball Data Overview:**

This data set with all games played in the Major League Baseball between the 2006 and 2016 season. Each row is a game with information on the home team, the visiting team, the season and the month of the game, etc.
We are interested in predicting the number of runs scored by the home team (`h_score`) using all the information on the game (teams, record, etc.) but not the runs scored by the opponent (`v_score`)---since that will be only available after the game has been played.

In [ ]:
### Import needed packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import regex
from datetime import datetime
from itertools import zip_longest,product

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import PoissonRegressor
from sklearn.linear_model import GammaRegressor

import statsmodels.api as sm

# Test for heteroscedasticity in OLS models
from statsmodels.stats.diagnostic import het_breuschpagan

In [ ]:
#pandas setting
pd.set_option('display.max_columns', None) ### always show me all columns
pd.set_option('display.max_rows', None) ### always show all rows
pd.set_option('display.float_format', '{:.2f}'.format) ### Turn off scientific notation (format float)

### Load Data from GitHub Repo:

In [ ]:
!git clone https://github.com/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning.git

In [ ]:
### Read Baseball Game Historical Data into DF
BBall_DF = pd.read_csv("/content/MSDIA_PredictiveModelingAndMachineLearning/GB886_III_9_BBallLogs.csv")

### Preliminary Data Analysis:

#### Descriptive Analysis:

In [ ]:
### View Dimensions of DF
BBall_DF.shape

In [ ]:
### View Datatypes
BBall_DF.info()

**<u>Notes:</u>**

day_of_week, h_name, and v_name are our object datatype fields,<br> everything else is of int64

In [ ]:
### View First Rows of data
BBall_DF.head()

In [ ]:
### View Descriptive Stats on Data
BBall_DF.describe()

In [ ]:
#### Check for null values
BBall_DF.isnull().sum()

In [ ]:
### View Total # of counts of each field variant within each obj variable
obj_data = BBall_DF.select_dtypes(include = ['object'])

for field in obj_data:
  print(BBall_DF[field].value_counts())

In [ ]:
#### Distriubtion of the h_score Field
BBall_DF['h_score'].value_counts()

####**Visual Analysis:**

**View Various Relationships between h_score and other variables in dataset:**

In [ ]:
### Variable Lists
x = [i for i in BBall_DF if i not in ['h_score','v_score','day_of_week','h_name','v_name']] ### Pull Relevant non object type x variables

In [ ]:
### View Distributions of a X Vars (Sans Object Types)
for xvar in x:
  print()

  sns.histplot(
      data = BBall_DF,
      x = f"{xvar}"
  )

  plt.title(f"Distribution of {xvar}")

  plt.show()

Notes:

A bulk of the games seem to be occuring on either of the 1st or 30th of each given month.





**View how values of Yi (h_score) are distributed:**

In [ ]:
### View Distribtion of Y Var (Total Runs scored by Home Team)
plt.hist(BBall_DF['h_score'])
plt.title("Distibution of h_score")
plt.show()

In [ ]:
### View Distribtion of Y Var (Total Runs scored by Home Team) (View with skinnier bars)
sns.histplot(
    data = BBall_DF,
    x = 'h_score',
)

plt.title("Distibution of h_score")
plt.show()

**<u>Notes:</u>**

Judging from the above chart, the h_score variable is discrete, which would ultimatley make it fall under the realm of a possion distribution, albeat 1 that is skewed to the right.

### **Modeling (Prep/Fit):**

#### **Variable Prep:**

In [ ]:
### Set Y Var
Y_Freq = BBall_DF['h_score']

### Set X Var
X = BBall_DF.select_dtypes(include = ['int64']).drop(columns = ['h_score','v_score']) ### Drop Categoricals, v_score, h_score variable

In [ ]:
Y_Freq.head()

In [ ]:
X.head()

In [ ]:
### Convert categorical variables to dummy variables
dummy_fields = pd.get_dummies(
    obj_data,
    dtype = int, ### Set Dummy Variable Data Type to Integer
    drop_first = True ### Drop First of each dummy variable to avoid dummy variable trap
)

### View Dummy Fields
dummy_fields.head()

In [ ]:
### Add Back Dummy Variables to original Dataframe
X_W_Dummies = pd.concat([X,dummy_fields], axis = 1) ### axis = 0: row-wise concat, axis = 1: column-wise concat

X_W_Dummies.head()

In [ ]:
### Check Shape of new X Var DF
X_W_Dummies.shape

In [ ]:
### Add Constant To Design Matrix
X_W_DummiesWConstant = sm.add_constant(X_W_Dummies)

X_W_DummiesWConstant

#### **Fit Model:**

In [ ]:
### Fit a Poission Regression on the data
# Create a PoissonRegressor Object
TotalRunsFreqModel = PoissonRegressor(max_iter = 10000)
# Fit model
TotalRunsFreqModel.fit(X_W_DummiesWConstant,Y_Freq)
# Predict based on X Variables
preds_freq = TotalRunsFreqModel.predict(X_W_DummiesWConstant)
# Show Correlations between Predictions and Outcomes
np.corrcoef(preds_freq,Y_Freq) ### about 12.62% Correlation Between Prediction and Outcomes

In [ ]:
### Generation of Model Summary for Further Analysis
link_p = sm.genmod.families.links.log  # use a log link
model_poisson = sm.GLM(Y_Freq, X_W_DummiesWConstant.astype(float), family=sm.families.Poisson(link_p())).fit()
print(model_poisson.summary())

**<u>Inital Model Notes:</u>**

There was an initial "Convergence error" when attempting to fit the model, so I had to increase the max interations threshold to 10000 to allow the algorithm to find an optimal solution.

The correlation coefficient between the predicted and actual Y Frequencies sits at about **12.62%.**

When looking at the model summary. We see that the Pseudo R-squared sits at 0.06333, indicating that only 6.3% of the variance is being explained by the model. The deviance score (an indicator of goodness of fit) sits at 55237 (less is better). This indicates that the model may not fit well on the data. The Chi-squared value of 5.31e+04 indicates the values for Y may be widley dispersed (variance > mean).
<br>
<br>
**<u>Interpretation of Beta Coefficients</u>:**

Beta Coeff for h_wins = 0.0019.

If we take e^0.0019 (plugged beta into log link function), we get 1.0019.
* 1 - 1.0019 = 0.0019.

We are saying here that for every unit change in h_wins, our predicted h_score increases by about 0.19%.

#### **Plot of Predicited Values vs Actuals:**

In [ ]:
### Plotting Predicted Freqs vs Actual Y Freqs
plt.scatter(Y_Freq,preds_freq)
plt.xlabel('Y_Freq')
plt.ylabel('Y_Preds_Freq')
plt.title("Y vs Predicted Y")
plt.show()

**Inital Observations**

Generally, the model seems to be underpredicting the total number of runs, especially as we move further and further to the left of the chart to higher Actual Frequencies. When we hit 20 for actual frequency of Runs, the model is predicting anywhere from 4.2 to 5.3 runs. It can be noted too that as we move to higher frequency levels, the predicted bands tend to get narrower, indicating that the model is more consistently assigning a predicted value for total runs (albiet still underpredicting).

#### **Further Validation:**

In [ ]:
print(f"Mean of Ys: {np.mean(Y_Freq)}")

In [ ]:
print(f"Variance of Ys: {np.var(Y_Freq)}")

**<u>Note:</u>**

Since the Variance of the Ys of this dataset exceed that of it's mean, this indicates overdispersion of the data, which can lead to potentially innacurate predictions. This validates the output for Chi-Squared in the model summary.

#### **Fit Model (Scaled Version):**

Will now perform standard scaling on the design matrix to see if this imporves the accuracy/fit of the model improves.

Standard scaling involves showing the normally distributed values (z-scores) of each of our predictor variables (mean of 0, stdev of 1), which accounts for cases like where we have a year field that it's in the 2000s vs having dummy variable fields (1/0). The model in that case may assign more inportance(higher weights) to the year field simply because it's a much large number than our dummy fields. Scaling puts both these fields on the same scale, allowing for more fair weighting.

In [ ]:
### Set Scaler Object
Scaler = StandardScaler()

In [ ]:
### Scale Design Matrix
X_W_Dummies_Scaled = Scaler.fit_transform(X_W_Dummies)

In [ ]:
### View Scale Design Matrix
X_W_Dummies_Scaled

In [ ]:
### Insert Back into a dataframe
X_W_Dummies_Scaled_DF = pd.DataFrame(X_W_Dummies_Scaled, columns = X_W_Dummies.columns)

In [ ]:
### Add Constant
X_W_Dummies_Scaled_WConstant = sm.add_constant(X_W_Dummies_Scaled_DF)

X_W_Dummies_Scaled_WConstant.head()

In [ ]:
### Fit a Scaled Poission Regression on the data
# Create a PoissonRegressor Object
TotalRunsFreqModel_Scaled = PoissonRegressor()
# Fit model
TotalRunsFreqModel_Scaled.fit(X_W_Dummies_Scaled_WConstant,Y_Freq)
# Predict based on X Variables
preds_freq_Scaled = TotalRunsFreqModel_Scaled.predict(X_W_Dummies_Scaled_WConstant)
# Show Correlations between Predictions and Outcomes
np.corrcoef(preds_freq_Scaled,Y_Freq) ### about 17.30% Correlation Between Prediction and Outcomes

In [ ]:
### Generation of Model Summary for Further Analysis
link_p = sm.genmod.families.links.log  # use a log link
model_poisson_scaled = sm.GLM(Y_Freq, X_W_Dummies_Scaled_WConstant.astype(float), family=sm.families.Poisson(link_p())).fit()
print(model_poisson_scaled.summary())

<u>**How to Interpret:**</u>

Beta Coeff for h_wins = 0.0469.

If we take e^0.0469 (plugged beta into log link function), we get 0.952.
* 1 - 0.952 = 4.8.

We are saying here that for 1 standard deviation change in the variable h_wins, our predict h_score increases by about 4.8%

#### **Scaled Plot of Predicted Versus Actuals:**

In [ ]:
### Plotting Predicted Freqs vs Actual Y Freqs
plt.scatter(Y_Freq,preds_freq_Scaled)
plt.xlabel('Y_Freq')
plt.ylabel('preds_freq_Scaled')
plt.title("Y vs Scaled Predicted Y")
plt.show()

**<u>Notes:</u>**

* The scaled model did not produce a "convergence error" when being fit, so no max iterations adjustment was needed.


* The Scaled Model yields an imporved correlation coefficient between y_predicted and y_acutal (17.3%)

* When looking at the regression summary, we actually see that the summary statistics such as the chi-squared and the deviance show the same values as in the unscaled model.

* When plotting the predicited results of our Scaled Model vs Actual Frequencies, we do see still see general underpredicting of total runs, but we do notice that the predicited values for y are slightly more condensed then our unscaled model as we move further to the right of the chart.

### **Predicting Total Runs in a Game:**

Will do so using the uscaled and scaled models to see if there are any significant differences between the 2

In [ ]:
### Read Data into Dataframe
Astros_DF = pd.read_csv('/content/AstrosSeason2017.csv', encoding='cp1252')

In [ ]:
### View DF
Astros_DF.head()

In [ ]:
### Keep only relavent Rows
Astros_DF_Modeling = Astros_DF.copy()

Astros_DF_Modeling.drop(columns = ['Unnamed: 2', 'Unnamed: 4', 'W/L','GB','Win','Loss','Save','Time','D/N','Attendance','Streak','Orig. Scheduled','Inn','Rank','R','RA','cLI'], inplace = True)

Astros_DF_Modeling.head()

In [ ]:
### Add Year and Team Name Field
Astros_DF_Modeling['Year'] = 2017

Astros_DF_Modeling.head()

In [ ]:
### Split Win/Loss Field
Astros_DF_Modeling[['W','L']] = Astros_DF_Modeling['W-L'].str.split('-',expand = True)
### Drop Old W-L Field
Astros_DF_Modeling.drop(columns = ['W-L'], inplace = True)

In [ ]:
# Clean the Date column first: remove leading/trailing spaces
Astros_DF_Modeling['Date'] = Astros_DF_Modeling['Date'].str.strip()

# Function to extract month and day
def extract_month_day(date_str):
    # Remove any extra non-date characters if needed
    date_str = date_str.split('(')[0].strip()  # removes anything after '('
    dt = datetime.strptime(date_str, '%A %b %d')
    return pd.Series([dt.month, dt.day])

# Apply function
Astros_DF_Modeling[['Month', 'Day']] = Astros_DF_Modeling['Date'].apply(extract_month_day)

# View DF
Astros_DF_Modeling.head()

In [ ]:
### Create a "day_of_week" Field
#Clean out any parentheses or trailing spaces
Astros_DF_Modeling['Date_clean'] = Astros_DF_Modeling['Date'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

#Add the year
Astros_DF_Modeling['Date_full'] = Astros_DF_Modeling['Date_clean'] + ' 2017'

#Parse into datetime (let pandas infer format)
Astros_DF_Modeling['Date_parsed'] = pd.to_datetime(Astros_DF_Modeling['Date_full'], errors='coerce')

#Create an Entry Map
day_map = {
    'Monday': 'Mon',
    'Tuesday': 'Tue',
    'Wednesday': 'Wed',
    'Thursday': 'Thu',
    'Friday': 'Fri',
    'Saturday': 'Sat',
    'Sunday': 'Sun'
}

Astros_DF_Modeling['day_of_week'] = Astros_DF_Modeling['Date_parsed'].dt.day_name().map(day_map)

Astros_DF_Modeling[['Date', 'day_of_week']].head()

In [ ]:
# Remove Old Date Field
Astros_DF_Modeling.drop(columns = ['Date','Date_full','Date_clean','Date_parsed'], inplace = True)

In [ ]:
# View DF
Astros_DF_Modeling.head()

In [ ]:
# Column Mover Function
def ColumnMover (df,colname, position):
  col = df.pop(colname)
  df.insert(position, colname, col)

In [ ]:
### Create a v_game_number field based off home game number
Astros_DF_Modeling['v_game_number'] = Astros_DF_Modeling['Gm#']

In [ ]:
### Move Columns to appropriate positions
ColumnMover(Astros_DF_Modeling,'Year',0)
ColumnMover(Astros_DF_Modeling,'Month',1)
ColumnMover(Astros_DF_Modeling,'Day',2)
ColumnMover(Astros_DF_Modeling,'Gm#',3)
ColumnMover(Astros_DF_Modeling,'W',4)
ColumnMover(Astros_DF_Modeling,'v_game_number',5)
ColumnMover(Astros_DF_Modeling,'L',6)
ColumnMover(Astros_DF_Modeling,'day_of_week',7)

In [ ]:
### Rename Fields Where Appropriate
Astros_DF_Modeling.rename(
    columns= {
        'Gm#' : 'h_game_number',
        'W': 'h_wins',
        'L': 'v_wins',
        'Tm': 'h_name',
        'Opp': 'v_name'
    }, inplace = True
)

In [ ]:
### View DF
Astros_DF_Modeling.head()

In [ ]:
### View Data Types
Astros_DF_Modeling.info()

In [ ]:
### Convert h_wins and v_wins to int64
Astros_DF_Modeling['h_wins'] = Astros_DF_Modeling['h_wins'].astype('int64')
Astros_DF_Modeling['v_wins'] = Astros_DF_Modeling['v_wins'].astype('int64')

In [ ]:
### Create Dummy Vars
Obj_Variables = Astros_DF_Modeling.select_dtypes(include = 'object')

Astros_Dummy_Vars = pd.get_dummies(
    Obj_Variables,
    dtype = int,
    drop_first = True
)

Astros_Dummy_Vars.head()

In [ ]:
### Drop Categ Fields from Modeling DF
Astros_DF_Modeling.drop(columns = ['day_of_week','h_name','v_name'], inplace = True)

### Merge Dummies Back with Modeling DF
Astros_DF_Modeling_wDummies = pd.concat([Astros_DF_Modeling,Astros_Dummy_Vars], axis = 1)

In [ ]:
### Add Constant
Astros_DF_Modeling_wDummies_Const = sm.add_constant(Astros_DF_Modeling_wDummies, has_constant='add')

In [ ]:
### View Modeling DF
Astros_DF_Modeling_wDummies_Const.head()

**<u>Note:</u>**

In order to utilize the fitted model, we need to ensure that we have the same amount of fields in this test data set that we used to train the model on.

In [ ]:
### Choose Specific Game from testing dataset
Chosen_Game = Astros_DF_Modeling_wDummies_Const.head(1)

Chosen_Game ### Hous Vs Seahawks

In [ ]:
#### List out all matchups from original dataset
field_list_model =X_W_DummiesWConstant.columns.to_list()

Team_Matchup_lst = [i for i in field_list_model if i.startswith('v_name') or i.startswith('h_name')]

Team_Matchup_lst

In [ ]:
### Store In a Dataframe
All_H_and_V_Df = pd.DataFrame(columns = Team_Matchup_lst)

In [ ]:
# Create a new row with all zeros
new_row = pd.Series(0, index=All_H_and_V_Df.columns)

# Set matchup teams to 1
new_row['h_name_HOU'] = 1
new_row['v_name_SEA'] = 1

# Add the new row to the DataFrame
All_H_and_V_Df.loc[len(All_H_and_V_Df)] = new_row

# All_H_and_V_Df

In [ ]:
### Drop All Matchup info from chosen df
Chosen_Game_Dropped = Chosen_Game.drop(columns = [i for i in Chosen_Game if i.startswith('v_name') or i.startswith('h_name')])

Chosen_Game_Dropped

In [ ]:
#### Merge stats from testing dataset with full matchup list
Chosen_Game_Final = pd.concat([Chosen_Game_Dropped,All_H_and_V_Df], axis = 1)

In [ ]:
### View Final Game DF Stats for Chosen Game
Chosen_Game_Final

#### **Unscaled Result:**

In [ ]:
### Run Unscaled Model
Predicted_Total_Runs = TotalRunsFreqModel.predict(Chosen_Game_Final)

print(f"Predicted Runs: {Predicted_Total_Runs}")

In [ ]:
### Compare to actual Result for this game
Actual_Runs = Astros_DF.iloc[0]['R']

print(f"Actual Runs: {Actual_Runs}")

The unscaled Model predicts a total runs amount of 4.09 for the Astros vs the Seattle. The actual result was 3.

####**Scaled Result:**

In [ ]:
### Scale Design Matrix
Chosen_Game_Final_Scaled = Scaler.fit_transform(Chosen_Game_Final)

In [ ]:
### Run Scaled Model
Predicted_Total_Runs_Scaled = TotalRunsFreqModel_Scaled.predict(Chosen_Game_Final_Scaled)

print(f"Predicted Runs (Scaled): {Predicted_Total_Runs_Scaled}")

Interestingly Enough, the scaled model provided a less accurate result, prediciting 4.5 runs.

<u>Main conclusion</u>:

The Models did a commendable job, but in the end..... Predicting stuff to happen in sports is hard...... (: